In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import warnings
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import platform

warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cuda')

In [2]:
CFG = {
    'TRAIN_WINDOW_SIZE':90, # 90일치로 학습
    'PREDICT_SIZE':21, # 21일치 예측
    'EPOCHS':10,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE': 2048,
    'SEED':41
}

PATH = os.getcwd() + '/data/'
if platform.system() == 'Darwin':
    LOADPATH = '/Users/a1r/Desktop/DL/timeseries_new_data/'
else:
    LOADPATH = '/home/a1r/바탕화면/DL/timeseries_new_data/'

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['SEED']) # Seed 고정

In [4]:
class GRU(nn.Module):
    def __init__(self, in_channel=9, hidden_size = 512, out_channel=CFG['PREDICT_SIZE']):
            super(GRU, self).__init__()
            self.hidden_size = hidden_size
            self.input_size = in_channel

            self.gru = nn.GRU(input_size = self.input_size,
                        hidden_size = self.hidden_size,
                        num_layers = 1,
                        bias=True,
                        bidirectional=False,
                        batch_first=True)
            
            self.fc = nn.Sequential(
                  nn.Linear(hidden_size, hidden_size//2),
                  nn.ReLU(),
                  nn.Dropout(),
                  nn.Linear(hidden_size//2, out_channel)
                  )
            self.actv = nn.ReLU()
    
    def forward(self, x):
        # x shape: (B: batch_size, TRAIN_WINDOW_SIZE: 90, 9)
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size, x.device)
        
        # gru layer
        gru_out, hidden = self.gru(x, hidden)
        
        # Only use the last output sequence
        last_output = gru_out[:, -1, :]
        
        # Fully connected layer
        output = self.actv(self.fc(last_output))
        
        return output.squeeze(1)
    
    def init_hidden(self, batch_size, device):
        # Initialize hidden state and cell state
        return Variable(torch.zeros(1, batch_size, self.hidden_size, device = device)) # h0

In [ ]:
# torch.Size([2048, 90, 9]) : input_size

# print(Variable(torch.zeros(1, 5, 10, device = device)).shape)
# Variable(torch.zeros(1, 5, 10, device = device))

## Training Dataset

In [5]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        
    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])
    
    def __len__(self):
        return len(self.X)

#### Call the Dataset Array

In [6]:
train_input = np.load(LOADPATH + 'train_input_minmax.npy')
train_target = np.load(LOADPATH + 'train_target_minmax.npy')

In [7]:
print(train_input.shape, train_target.shape)
print(train_input[:2])

(5132470, 90, 9) (5132470, 21)
[[[ 1.      6.     37.     ...  0.8413  1.      0.    ]
  [ 1.      6.     37.     ...  0.914   0.      0.    ]
  [ 1.      6.     37.     ...  1.45    0.      0.    ]
  ...
  [ 1.      6.     37.     ...  0.522   1.      0.    ]
  [ 1.      6.     37.     ...  0.653   1.      0.    ]
  [ 1.      6.     37.     ...  0.8267  1.      0.0833]]

 [[ 1.      6.     37.     ...  0.914   0.      0.    ]
  [ 1.      6.     37.     ...  1.45    0.      0.    ]
  [ 1.      6.     37.     ...  2.422   0.      0.    ]
  ...
  [ 1.      6.     37.     ...  0.653   1.      0.    ]
  [ 1.      6.     37.     ...  0.8267  1.      0.0833]
  [ 1.      6.     37.     ...  1.465   1.      0.    ]]]


In [8]:
data_len = len(train_input)
train_dataset = CustomDataset(train_input[:-int(data_len*0.2)], train_target[:-int(data_len*0.2)])
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(train_input[-int(data_len*0.2):], train_target[-int(data_len*0.2):])
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [9]:
def train(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    best_loss = 9999999
    best_model = None
    train_loss = {}
    val_loss = {}
    
    # Epoch
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss_li = []
        train_mae = []
    
    # Iteration
        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)
            
            optimizer.zero_grad()
            
            output = model(X)
            loss = criterion(output, Y)
            
            loss.backward()
            optimizer.step()
            
            train_loss_li.append(loss.item())
        
        val_loss_li = validation(model, val_loader, criterion, device)
        val_loss_mean = np.mean(val_loss_li)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss_li):.5f}] Val Loss : [{val_loss_mean:.5f}]')
        
        if best_loss > val_loss_mean:
            best_loss = val_loss_mean
            best_model = model
            print('Model Saved')

        train_loss[epoch] = train_loss_li
        val_loss[epoch] = val_loss_li

    return best_model, train_loss, val_loss

In [10]:
def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []
    
    with torch.no_grad():
        for X, Y in tqdm(iter(val_loader)):
            X = X.to(device)
            Y = Y.to(device)
            
            output = model(X)
            loss = criterion(output, Y)
            
            val_loss.append(loss.item())
    return val_loss

In [11]:
model = GRU()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG['LEARNING_RATE'])
infer_model, train_loss, val_loss = train(model, optimizer, train_loader, val_loader, device)

  0%|          | 0/2005 [00:00<?, ?it/s]

  0%|          | 0/502 [00:00<?, ?it/s]

Epoch : [1] Train Loss : [0.03074] Val Loss : [0.03178]
Model Saved


  0%|          | 0/2005 [00:00<?, ?it/s]

  0%|          | 0/502 [00:00<?, ?it/s]

...

  0%|          | 0/502 [00:00<?, ?it/s]

Epoch : [9] Train Loss : [0.01812] Val Loss : [0.01780]


  0%|          | 0/2005 [00:00<?, ?it/s]

  0%|          | 0/502 [00:00<?, ?it/s]

Epoch : [10] Train Loss : [0.01802] Val Loss : [0.01871]


In [12]:
SAVEPATH = os.getcwd() + '/GRU_1e_4.pth'
torch.save(infer_model.state_dict(), SAVEPATH)

train_loss = np.array(train_loss)
val_loss = np.array(val_loss)

# loss값 저장
np.save(PATH + 'train_loss_gru', train_loss)
np.save(PATH + 'val_loss_gru', val_loss)

In [13]:
help(nn.GRU)

Help on class GRU in module torch.nn.modules.rnn:

class GRU(RNNBase)
 |  GRU(*args, **kwargs)
 |  
 |  Applies a multi-layer gated recurrent unit (GRU) RNN to an input sequence.
 |  
 |  
 |  For each element in the input sequence, each layer computes the following
 |  function:
 |  
 |  .. math::
 |      \begin{array}{ll}
 |          r_t = \sigma(W_{ir} x_t + b_{ir} + W_{hr} h_{(t-1)} + b_{hr}) \\
 |          z_t = \sigma(W_{iz} x_t + b_{iz} + W_{hz} h_{(t-1)} + b_{hz}) \\
 |          n_t = \tanh(W_{in} x_t + b_{in} + r_t * (W_{hn} h_{(t-1)}+ b_{hn})) \\
 |          h_t = (1 - z_t) * n_t + z_t * h_{(t-1)}
 |      \end{array}
 |  
 |  where :math:`h_t` is the hidden state at time `t`, :math:`x_t` is the input
 |  at time `t`, :math:`h_{(t-1)}` is the hidden state of the layer
 |  at time `t-1` or the initial hidden state at time `0`, and :math:`r_t`,
 |  :math:`z_t`, :math:`n_t` are the reset, update, and new gates, respectively.
 |  :math:`\sigma` is the sigmoid function, and :math:`